In [ ]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings( 'ignore' )

In [ ]:
path = '/content/drive/MyDrive/dacon/airline/data/'
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

# 데이터 전처리
- EDA 기반으로 데이터 전처리를 진행한다. 

In [ ]:
# 변수 제거
train = train_df.drop(['id', 'Departure Delay in Minutes', 'Departure/Arrival time convenient'], axis=1)
test = test_df.drop(['id', 'Departure Delay in Minutes', 'Departure/Arrival time convenient'], axis=1)

In [ ]:
# 변수 변환

#log1p 변환
train['Arrival Delay in Minutes'] = np.log1p(train['Arrival Delay in Minutes'])
test['Arrival Delay in Minutes'] = np.log1p(test['Arrival Delay in Minutes'])

# 설문값 통합
train['Online support'][train['Online support'] == 1] = 3
train['Online support'][train['Online support'] == 2] = 3

train['Seat comfort'][train['Seat comfort'] == 4] = 5

train['Ease of Online booking'][train['Ease of Online booking'] == 1] = 3
train['Ease of Online booking'][train['Ease of Online booking'] == 2] = 3

test['Online support'][test['Online support'] == 1] = 3
test['Online support'][test['Online support'] == 2] = 3

test['Seat comfort'][test['Seat comfort'] == 4] = 5

test['Ease of Online booking'][test['Ease of Online booking'] == 1] = 3
test['Ease of Online booking'][test['Ease of Online booking'] == 2] = 3

In [ ]:
# 결측치 처리(만족도 == 0)
discrete_cols = list(set(train.columns) - {'Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes', 'target'})
categorical_cols = train.columns[train.dtypes == 'object']
satisfaction_cols = list(set(discrete_cols) - set(categorical_cols))

satistsfaction_0 = pd.DataFrame()
for col in satisfaction_cols:
  df = train[train[col] == 0]
  satistsfaction_0 = pd.concat([df, satistsfaction_0])
satistsfaction_0.drop_duplicates(inplace = True)

train.drop(satistsfaction_0.index, inplace = True)


In [ ]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [ ]:
# 파생변수
# 출발/도착 지연 여부 변수

train['Delay yn'] = 1
train['Delay yn'][train['Arrival Delay in Minutes'] == 0] = 0
train['Delay yn'][train_df['Departure Delay in Minutes'] == 0] = 0

test['Delay yn'] = 1
test['Delay yn'][test['Arrival Delay in Minutes'] == 0] = 0
test['Delay yn'][test_df['Departure Delay in Minutes'] == 0] = 0

In [ ]:
train

,Age,Flight Distance,Seat comfort,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,target,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,Delay yn
0,22,1599,3,3,3,4,3,4,4,5,4,4,4,5,4,0.000000,0,1,0,0,1,1,0,0,1,0,0
1,37,2810,2,4,4,1,4,3,5,5,4,2,1,5,2,2.944439,0,1,0,1,0,1,0,1,0,0,1
2,46,2622,1,1,1,4,5,5,4,4,4,4,5,4,3,0.000000,1,0,1,1,0,1,0,1,0,0,0
3,24,2348,3,3,3,3,3,3,3,2,4,5,3,4,3,1.098612,0,1,0,0,1,1,0,0,1,0,1
4,58,105,3,3,3,4,4,5,4,4,4,4,4,4,5,0.000000,1,1,0,1,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,30,2377,1,1,1,1,4,4,3,3,5,3,4,2,4,5.420535,0,0,1,1,0,0,1,0,1,0,1
2996,24,1643,2,3,4,5,3,5,5,2,2,4,1,3,5,2.639057,0,1,0,0,1,1,0,0,1,0,1
2997,22,1442,2,2,3,4,2,4,4,3,2,3,4,3,4,4.219508,0,1,0,0,1,1,0,0,1,0,1
2998,33,2158,2,2,5,4,2,4,4,5,2,5,5,5,4,1.386294,0,1,0,0,1,1,0,1,0,0,0


In [ ]:
test

,Age,Flight Distance,Seat comfort,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,Delay yn
0,61,2037,1,1,1,3,5,5,5,5,5,5,3,5,3,4.077537,1,0,1,0,0,1,0,1,0,1
1,27,1846,1,1,4,1,1,3,3,3,4,5,4,4,1,0.000000,1,0,0,1,1,0,1,0,0,0
2,52,1622,5,4,4,4,3,4,5,5,5,5,4,5,3,0.000000,1,0,1,0,1,0,1,0,0,0
3,54,3534,5,4,4,5,4,5,3,2,2,2,5,2,1,0.000000,0,1,1,0,1,0,1,0,0,0
4,41,1471,5,3,3,4,4,4,4,3,1,2,5,4,4,0.000000,1,0,1,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,62,54,3,1,1,4,3,4,3,3,3,3,3,3,1,0.000000,1,0,1,0,1,0,0,0,1,0
1996,33,1937,5,4,4,3,5,5,5,5,5,5,1,5,3,4.605170,1,0,1,0,1,0,1,0,0,1
1997,29,2398,3,4,4,3,3,3,3,1,2,3,4,2,3,1.386294,0,1,1,0,1,0,0,1,0,0
1998,51,406,5,5,4,4,3,3,3,2,5,2,2,2,2,4.430817,1,0,1,0,0,1,1,0,0,1


# 모델링

In [ ]:
train_y = train['target']

train.drop('target', axis=1, inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 0, n_estimators = 100)
rf.fit(train, train_y)
submission['target'] = rf.predict(test)

submission.to_csv(path + 'airline_1.csv', index=False)